last working app without theme, icon or increased confidence

In [1]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox

record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window


# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x200") # size of the settings window
    
    tk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = tk.Frame(settings_window)
    fps_frame.pack(pady=10)

    tk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = tk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)
    # buffer time option
    buffer_frame = tk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    tk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = tk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)


    def save_settings():
        global selected_fps
        try:
            selected_fps = int(fps_spinbox.get())
            global record_buffer_seconds
            record_buffer_seconds = int(buffer_spinbox.get())
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")
            
    bf = tk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "weapon" in label or "gun" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

In [2]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
from playsound import playsound


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="logo.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    # import threading
    # from playsound import playsound

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 10 second means alarm will sound after 10 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                threading.Thread(target=lambda: playsound("siren alert.mp3"), daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()


# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()